In [57]:
# Import libraries
import io
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error          # Function to calculate the Mean Squared Error (our performance measure)
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split    # Function for splitting dataset into train and test


# Load data 
url = 'https://drive.switch.ch/index.php/s/37RuoA3Mgt9Rqah/download'
response = requests.get(url)
data = np.load(io.BytesIO(response.content))


# Alternatively you can load the data from file
#data_path = 'data.npz' # path to the .npz file storing the data
#data = np.load(data_path)

# x is a Numpy array of shape (n_samples, n_features) with the inputs
x = data.f.x
# y is a Numpy array of shape (n_samples, ) with the targets
y = data.f.y

# T1
# train and test split
def f(x, theta):
    out = theta[0] + theta[1]*x[1] + theta[2]*x[2] + theta[3]*cos(x[1]) + theta[4]*x[2]**2 + theta[5] * tanh(x[1])
    return out

train, test = train_test_split(data.f.x, train_size=0.7, shuffle=True, random_state=0)

x_train = train[:, 0:6]
y_train = train[:, -1]

x_test = test[:, 0:6]
y_test = test[:, -1]


features_vector = x[:,0].reshape(-1,1)
ones_vector = np.ones(shape=(len(features_vector),1))
X = np.hstack((ones_vector, features_vector))

# method 1
theta_hat = np.linalg.solve(a=X.T.dot(X), b=X.T.dot(y))
print('Optimal parameters:\t', theta_hat)

# method 2, with linreg library
lr = LinearRegression(fit_intercept=False)
lr.fit(X, y)
theta_hat2 = lr.coef_
print('theta_hat = \t{}'.format(theta_hat))
print('theta_hat2 = \t{}'.format(theta_hat2))

print("theta: ")
print("Mean squared error: ")

# T2

print("Everything useful")

# T3 (Bonus)

print("Everything useful")

Input (X) shape: (4000, 2)
Optimal parameters:	 [21.12950315  3.64229851]
theta_hat = 	[21.12950315  3.64229851]
theta_hat2 = 	[21.12950315  3.64229851]
theta: 
Mean squared error: 
Everything useful
Everything useful


In [ ]:
# Import libraries
import joblib
import io
import requests
import torch
import numpy as np

def evaluate_predictions(y_true, y_pred):
    """
    Evaluates the mean squared error between the values in y_true and the values
    in y_pred.
    ### YOU CAN NOT EDIT THIS FUNCTION ###
    :param y_true: Numpy array, the true target values from the test set;
    :param y_pred: Numpy array, the values predicted by your model.
    :return: float, the mean squared error between the two arrays.
    """
    assert y_true.shape == y_pred.shape
    return ((y_true - y_pred) ** 2).mean()


def load_model(filename):
    """
    Loads a torch model saved.
    This is just an example, you can write your own function to load the model.
    Some examples can be found in src/utils.py.
    :param filename: string, path to the file storing the model.
    :return: the model.
    """
    model = torch.jit.load(filename)

    return model

# Load the data
# This will be replaced with our private test data when grading the assignment

# Load data from url
url = 'https://drive.switch.ch/index.php/s/Wp0I2gb33mhERFN/download'
response = requests.get(url)
data = np.load(io.BytesIO(response.content))

# Alternatively yo can load the data from file
#data_path = 'data_bonus_test.npz'
#data = np.load(data_path)

# x is a Numpy array of shape (n_samples, n_features) with the inputs
x = torch.tensor(data.f.x, dtype=torch.float32)
# y is a Numpy array of shape (n_samples, ) with the targets
y =  torch.tensor(data.f.y,dtype=torch.float32).reshape(-1, 1)

# Load the trained model
baseline_model_path = 'baseline.pt'
baseline_model = load_model(baseline_model_path)

# Predict on the given samples
y_pred_ours = baseline_model(x)

############################################################################
# STOP EDITABLE SECTION: DO NOT modify anything above this point.
############################################################################

############################################################################
# ADD HERE YOUR CODE TO READ MODEL OF TASK 3
############################################################################

# Load the trained model
baseline_model_path = 'YOUR_MODEL_PATH'
baseline_model =  ...   # LOAD YOU MODEL and predict x
# Predict on the given samples FROM YOUR MODEL
y_pred_yours = ...


############################################################################
# STOP EDITABLE SECTION: do not modify anything below this point.
############################################################################

# Evaluate the prediction using MSE
mse = evaluate_predictions(y_pred_yours, y)
print(f'MSE on whole dataset: {mse}')

# NOTE: NOW THIS CELL IS NOT WORKING SINCE YOU NEED TO CHANGE THE INPUT.
# DO IT AND EVERYTHING RUNS SMOOTH